# DB2-Salesforce connector: pull in new email flags from DB2

In [2]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"shit9927289sSYTkLiGvyK2UrazHFmjFUGU",#GqnyixN6W3zVDAFXk80lgMAR4
} 

In [3]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Get the latest email opt in/out data from DB2


In [4]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))

# get new preferences for users
pcomms = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles \
where profile_key in ('personalizedcommunication') order by user_id desc limit 5000;",engine)

up_news = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles \
where profile_key in ('updates_news') order by user_id desc limit 5000;",engine)


In [6]:
display(pcomms.head(2))
display(pcomms.tail(2))
display(up_news.head(2))
display(up_news.tail(2))

,id,user_id,profile_key,profile_value
0,1986689,297965,personalizedcommunication,"No, I do not want to receive personalized comm..."
1,1986694,297964,personalizedcommunication,"Yes, I would like to receive personalized comm..."


,id,user_id,profile_key,profile_value
4998,1949481,291327,personalizedcommunication,"Yes, I would like to receive personalized comm..."
4999,1949479,291326,personalizedcommunication,"Yes, I would like to receive personalized comm..."


,id,user_id,profile_key,profile_value
0,1986690,297965,updates_news,"No, I do not want to receive updates and news"
1,1986695,297964,updates_news,"Yes, I would like to receive updates and news"


,id,user_id,profile_key,profile_value
4998,1949482,291327,updates_news,"No, I do not want to receive updates and news"
4999,1949480,291326,updates_news,"Yes, I would like to receive updates and news"


In [ ]:
## adjusting new_email dataframe for the new email optin values 
# 0 or 1
pcomms_t = pcomms[['user_id','profile_value']]
up_news_t = up_news[['user_id','profile_value']]

display(pcomms_t.head(2))
display(up_news_t.head(2))


In [ ]:
## filtering pcomms_t with 0 or 1
ptt = pcomms_t['profile_value'].to_list()

for index, val in enumerate(ptt):
    val = val.split(',')[0]
    if val == 'Yes':
        ptt[index] = 1 # 1 indicates opt-in
    else:
        ptt[index] = 0 # 1 indicates opt-out

pcomms_t.loc[:,'profile_value'] = ptt


In [ ]:
## filtering up news t with 0 or 1
untt = up_news_t['profile_value'].to_list()

for index, val in enumerate(untt):
    val = val.split(',')[0]
    if val == 'Yes':
        untt[index] = 1 # 1 indicates opt-in
    else:
        untt[index] = 0 # 1 indicates opt-out

up_news_t.loc[:,'profile_value'] = untt


In [ ]:
## display
display(pcomms_t.head(2))
display(pcomms_t.tail(2))
display(up_news_t.head(2))
display(up_news_t.tail(2))

In [ ]:
## create new dataframe
new_opts = pd.DataFrame()

new_opts['nanoHUB_user_ID__c'] = pcomms_t['user_id']
new_opts['personalizedcommunication__c'] = pcomms_t['profile_value']
new_opts['updatesnews__c'] = up_news_t['profile_value']


In [ ]:
display(new_opts.head(2))
print(type(new_opts))

## Send to SF

In [ ]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)


In [ ]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(new_opts)


In [ ]:
# check status
db_s.check_bulk_status()

In [ ]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())